In [3]:
import os
def delete_all_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                delete_all_files_in_folder(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [6]:
from collections import OrderedDict
from multiprocessing import Pool
from cutting_polygon import *
from shapely.geometry import Polygon
import os
import numpy as np
import json
# number of LabelMe2YOLO multiprocessing threads
NUM_THREADS = max(1, os.cpu_count() - 1)
FACENAME_LIST = ["negx", "posx", "negy","posy", "negz", "posz"]
FACEMAP_DICT = {"negx":[0,1,1,2],"posx":[2,3,1,2],"negy":[1,2,0,1],"posy":[1,2,2,3],"negz":[3,4,1,2],"posz":[1,2,1,2],}

def get_label_id_map(json_dir: str):
    '''Get label id map from json files in json_dir'''
    label_set = set()

    for file_name in os.listdir(json_dir):
        if file_name.endswith("json"):
            json_path = os.path.join(json_dir, file_name)
            data = json.load(open(json_path))
            for shape in data["annotations"]:
                label_set.add(shape["name"])
    
    label_list = []
    for label_id, label in [(label, label_id) for label_id, label in enumerate(label_set)]: 
        print(label_id, label)
        label_list.append((label_id+"_hole", label+len(label_set)))
        label_list.append((label_id, label))

    return OrderedDict(label_list)


class Darwin2YOLO(object):
    '''Darwin to YOLO format converter'''
    def __init__(self, json_dir, output_format, label_list):
        self._json_dir = json_dir
        self._output_format = output_format
        self._label_list = label_list

        if label_list:
            self._label_id_map = {label: label_id
                                  for label_id, label in enumerate(label_list)}
            #print(self._label_id_map)
        else:
            self._label_id_map = get_label_id_map(self._json_dir)
            print((self._label_id_map))
            self._label_list = [label for label in self._label_id_map.keys()]
            print((self._label_list))
            
    def convert(self):
        json_names = [file_name for file_name in os.listdir(self._json_dir)
                      if os.path.isfile(os.path.join(self._json_dir, file_name)) and
                      file_name.endswith('.json')]
        self.output_path = os.path.join(self._json_dir + '/output/')
        if os.path.exists(self.output_path):
            delete_all_files_in_folder(self.output_path)
        else:
            os.makedirs(self.output_path)

        # convert labelme object to yolo format object, and save them to files
        # also get image from labelme json file and save them under images folder
        pool = Pool(NUM_THREADS)

        for json_name in json_names:
            self.covert_json_to_text(self.output_path, json_name)
            #pool.apply_async(self.covert_json_to_text,
                                #args=(output_path, json_name))
        pool.close()
        pool.join()

        
    def covert_json_to_text(self, target_dir, json_name):
        json_path = os.path.join(self._json_dir, json_name)
        json_data = json.load(open(json_path))

        print('Converting %s for %s ...' %
              (json_name, target_dir.replace('/', '')))

        yolo_obj_dict = self._get_yolo_object_list(json_data)
        
        for facename in FACENAME_LIST:
            x = yolo_obj_dict.get(facename, None)
            #print(f"x = {x}")
            if x is not None:
                save_yolo_label(json_name[:-5] +"_"+ facename+".json",
                                self.output_path,
                                x)
        
    def _get_yolo_object_list(self, json_data):
        
        # initialize dictionary
        dict_yolo_obj_list = {}
        
        # iterating through the elements of list
        for i in FACENAME_LIST:
            dict_yolo_obj_list[i] = []

        #img_h, img_w, _ = cv2.imread(img_path).shape
        img_w = int(json_data["item"]["slots"][0]["width"])
        img_h = int(json_data["item"]["slots"][0]["height"])

        #for each object annotated
        for shape in json_data["annotations"]:

            yolo_obj_list = self._get_other_shape_yolo_object(
                shape, img_h, img_w)
            
            #for each polygon i: (each object can have more than 1 polygon)
            #   yolo_obj_list[i][0] = [class,points]
            #   yolo_obj_list[i][1] = [face]
            #   yolo_obj_list[i][0][0] = [class]
            #   yolo_obj_list[i][0][1] = [points]

            poly_list = [Polygon(yolo_obj_list[i][0][1]) for i in range(len(yolo_obj_list))]

            #print(f"len(yolo_obj_list) = {len(yolo_obj_list)}")
            #print(f"len(poly_list) = {len(poly_list)}")
            #print(f"")
            #print(type(yolo_obj_list[0][0]))
            #for yolo_obj, cubemap_faces in yolo_obj_list:
            for i in range(len(yolo_obj_list)):
                yolo_obj, cubemap_faces = yolo_obj_list[i]

                area_superposition_count = 0

                for polyb in poly_list:
                    if poly_list[i] != polyb:
                        if polyb.contains(poly_list[i]):
                            area_superposition_count+=1
                #print(f"area_superposition_count = {area_superposition_count}")

                if area_superposition_count%2==1:
                    yolo_obj[0] += len(self._label_list)//2
                    #print(f"yolo_obj_list[i][0][1] = {yolo_obj_list[i][0][1]}")
                    #print(f"is a hole")
                    #print(f"----------------------------------------------------")

                for cubemap_face in cubemap_faces:
                    x = dict_yolo_obj_list.get(cubemap_face, None)
                    if x is not None:
                        minX, maxX, minY, maxY = FACEMAP_DICT.get(cubemap_face,None)
                        
                        #TODO lembrar de tirar esse if depois
                        if cubemap_face == "NADA":
                            print(f"minY = {minY}")
                            print(f"maxY = {maxY}")
                            print(f"minX = {minX}")
                            print(f"maxX = {maxX}")
                            print(f"uncut polygon = {yolo_obj[1]}")
                        resulting_polygons = cut_polygon_between_2_horizontal_line(yolo_obj[1], minY, maxY)
                        resulting_polygons = cut_polygon_between_2_vertical_line(resulting_polygons, minX, maxX)
                        if cubemap_face == "NADA":
                            print(f"resulting_polygons = {resulting_polygons}")
                            print("")
                        if len(resulting_polygons) > 0:
                            x.append([yolo_obj[0]] + resulting_polygons)
                    else:
                        print("x ta sendo None, isso n devia estar acontecendo")

        return dict_yolo_obj_list
    
    def _get_other_shape_yolo_object(self, shape, img_h, img_w):

        yolo_obj_list = []        
        #point_list = shape["polygon"]["paths"][0]
        polygon_list = shape["polygon"]["paths"]

        for point_list in polygon_list:
            points = np.zeros((len(point_list),2))

            #points[::2] = [(float(point[0]) / img_w) if point[0]<=img_w else 1 for point in point_list]
            # limit of points are img_w and img_h
            #points[::2] = [1 if float(point["x"]) > img_w else 
            #               (0 if float(point["x"]) < 0 else 
            #                (float(point["x"]) / img_w)) for point in point_list]
            
            #points[1::2] = [(float(point[1]) / img_h) if point[1]<=img_h else 1 for point in point_list]
            #points[1::2] = [1 if float(point["y"]) > img_h else 
            #               (0 if float(point["y"]) < 0 else 
            #                (float(point["y"]) / img_h)) for point in point_list]

            points[:] = [[point_list[i]["x"]/ img_w, point_list[i]["y"]/ img_h] for i in range(len(point_list))]
            #points[::2] = points[::2]*4
            #points[1::2] = points[1::2]*3
            points.clip(0,1)
            #print(f"points = {points}")
            points = points * (4,3)
            cubemap_faces = self.get_cubemaps_faces(points[:, 0], points[:, 1])

            #print(f"shape['label'] = {shape['label']}")
            #print(f"_label_id_map = {self._label_id_map}")

            shape_label = shape.get('name',-1)
            #print(f"shape_label = {shape_label}")
            if shape_label in self._label_id_map:
                label_id = self._label_id_map[shape_label]
                #print(f"label_id = {label_id}")
                #print(f"")
            else:
                print(f"label {shape['name']} not in {self._label_list}")
                return -1
            #print(points)
            yolo_obj_list.append(([label_id, points.tolist()], cubemap_faces))
        return yolo_obj_list
    
    def get_cubemaps_faces(self, points_x, points_y):
        cubemaps_faces = []
        for (x,y) in zip(points_x, points_y):
            if x<=1:
                cubemaps_faces.append("negx") if "negx" not in cubemaps_faces else cubemaps_faces
            if x>=1 and x<=2:
                if y<=1:
                    cubemaps_faces.append("negy") if "negy" not in cubemaps_faces else cubemaps_faces
                if y>=1 and y<=2:
                    cubemaps_faces.append("posz")  if "posz" not in cubemaps_faces else cubemaps_faces
                if y>=2:
                    cubemaps_faces.append("posy") if "posy" not in cubemaps_faces else cubemaps_faces
            if x>=2 and x<=3:
                cubemaps_faces.append("posx") if "posx" not in cubemaps_faces else cubemaps_faces
            if x>=3:
                cubemaps_faces.append("negz") if "negz" not in cubemaps_faces else cubemaps_faces
            
        return cubemaps_faces
    
def save_yolo_label(json_name, output_path, yolo_obj_list):
    '''Save yolo label to txt file'''
    txt_path = os.path.join(output_path,
                            "cubemap_"+json_name.replace(".json", ".txt"))
    face = json_name[-9:-5]
    
    with open(txt_path, "w+") as f:
        for yolo_obj in yolo_obj_list:
            #print(f"yolo_obj ={yolo_obj}")
            label = yolo_obj[0]
            points = custom_modulo(flatten(yolo_obj[1:]), face)
            points = [str(item) for item in points]
            yolo_obj_line = f"{label} {' '.join(points)}\n"
            f.write(yolo_obj_line)

def custom_modulo(points, face):
    return_list = np.array(points)
    return_list[::2]=return_list[::2]-(FACEMAP_DICT.get(face,0)[0])
    return_list[1::2]=return_list[1::2]-(FACEMAP_DICT.get(face,0)[2])
    return return_list.tolist()

def flatten(l):
    return [item for sublist in l for item in sublist]


In [11]:
convertor = Darwin2YOLO(json_dir="SISTEMAS_UFF/corrosion-v4/", output_format="polygon",label_list=None)
convertor.convert()

Corrosão 0
OrderedDict([('Corrosão_hole', 1), ('Corrosão', 0)])
['Corrosão_hole', 'Corrosão']
Converting P74_01b88e07cc3ff7126288fc499d9c4ba4.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_db49b901d10c04bef7fa1a1fd0448dac.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_9c1a613db60312d3f1ba7b5c36044c94.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_fcf1ef699272142e54979be634513d81.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_764b2ec5a4038b7a934cc54be82aa580.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_8f542b4547d7e7034bda8ff839be3445.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_35182ee3e16b64dea83dda27a2951ccc.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_585e157b95823096d8a3415f64ab02a6.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_71dc91a34f2deb552cf7eaa1e4484975.json for SISTEMAS_UFFcorrosion-v4output ...
Converting P74_81bc6236508f38d6c14ed32a6133c32d.json for SISTEMAS_UFFcorrosion